# About
This notebook contains a first simple ML model

In [3]:
# Necessary to import custom modules
import os
os.chdir("/home/jovyan/work")

from neomodel import db
import pandas as pd

from src.utils import init_connection

In [4]:
init_connection()

In [4]:
cypher = """
MATCH 
    (t:Team)-[:SCORED]->(s:Score)-[:IN_GAME]->(g:Game)<-[:IN_GAME]-(s2:Score)<-[:SCORED]-(t2:Team), (sea:Season)
WHERE 
    (g)-[:TOOK_PLACE_IN]->(sea)
RETURN 
    t.name as team, 
    sum(CASE WHEN s.score > s2.score AND g.game_type = "regular_season" THEN 1 ELSE 0 END) as wins_reg,
    sum(CASE WHEN s.score < s2.score AND g.game_type = "regular_season" THEN 1 ELSE 0 END) as losses_reg,
    sea.name as season
ORDER BY 
    team, season
"""

In [5]:
data, columns = db.cypher_query(cypher)

In [7]:
df_results = pd.DataFrame(data=data, columns=columns)

In [27]:
df_test = df_results.pivot(index="team", columns="season")

In [28]:
cols = []
for a, b in zip(df_test.columns.get_level_values(0), df_test.columns.get_level_values(1)):
    cols.append(a+"_"+b)

In [29]:
cols

['wins_reg_2015/2016',
 'wins_reg_2016/2017',
 'wins_reg_2017/2018',
 'losses_reg_2015/2016',
 'losses_reg_2016/2017',
 'losses_reg_2017/2018']

In [31]:
df_test.columns = cols

In [32]:
df_test

,wins_reg_2015/2016,wins_reg_2016/2017,wins_reg_2017/2018,losses_reg_2015/2016,losses_reg_2016/2017,losses_reg_2017/2018
team,,,,,,
Atlanta Hawks,34,39,58,48,43,24
Boston Celtics,34,29,27,48,53,55
Brooklyn Nets,61,62,54,21,20,28
Charlotte Hornets,34,46,46,48,36,36
Chicago Bulls,40,41,55,42,41,27
Cleveland Cavaliers,25,31,32,57,51,50
Dallas Mavericks,40,49,58,42,33,24
Denver Nuggets,49,42,36,33,40,46
Detroit Pistons,38,45,43,44,37,39


#### Extract Matchup

```
MATCH 
	(t:Team)-[:SCORED]->(s:Score)-[:IN_GAME]->(g:Game)<-[:IN_GAME]-(s2:Score)<-[:SCORED]-(t2:Team), (sea:Season)
WHERE 
    (g)-[:TOOK_PLACE_IN]->(sea)
AND
	sea.name = "2017/2018"
AND
	g.game_type = "regular_season"
WITH g, collect(t.name) as teams, collect(s.score) as scores
RETURN
	HEAD(teams) as teamA, HEAD(TAIL(scores)) as scoreA, HEAD(TAIL(teams)) as teamB, HEAD(scores) as scoreB
```

In [35]:
cypher = """
MATCH 
	(t:Team)-[:SCORED]->(s:Score)-[:IN_GAME]->(g:Game)<-[:IN_GAME]-(s2:Score)<-[:SCORED]-(t2:Team), (sea:Season)
WHERE 
    (g)-[:TOOK_PLACE_IN]->(sea)
AND
	sea.name = "2017/2018"
AND
	g.game_type = "regular_season"
WITH g, collect(t.name) as teams, collect(s.score) as scores
RETURN
	HEAD(teams) as teamA, HEAD(TAIL(scores)) as scoreA, HEAD(TAIL(teams)) as teamB, HEAD(scores) as scoreB
"""

In [37]:
data, cols = db.cypher_query(cypher)

In [45]:
df_matches = pd.DataFrame(data=data, columns=cols)

In [52]:
df_matches["teamAWinner"] =df_matches["scoreA"] > df_matches["scoreB"]

In [58]:
df_test.head().reset_index()

,team,wins_reg_2015/2016,wins_reg_2016/2017,wins_reg_2017/2018,losses_reg_2015/2016,losses_reg_2016/2017,losses_reg_2017/2018
0,Atlanta Hawks,34,39,58,48,43,24
1,Boston Celtics,34,29,27,48,53,55
2,Brooklyn Nets,61,62,54,21,20,28
3,Charlotte Hornets,34,46,46,48,36,36
4,Chicago Bulls,40,41,55,42,41,27


In [55]:
df_matches.head()

,teamA,scoreA,teamB,scoreB,teamAWinner
0,Chicago Bulls,98,Toronto Raptors,122,False
1,Golden State Warriors,120,Washington Wizards,117,True
2,Philadelphia 76ers,116,Milwaukee Bucks,94,True
3,New York Knicks,98,Philadelphia 76ers,105,False
4,Sacramento Kings,100,San Antonio Spurs,107,False


In [107]:
df_ml = pd.merge(
    pd.merge(df_matches, df_test.reset_index(), left_on="teamA", right_on="team"),
    df_test.reset_index(), 
    left_on="teamB", 
    right_on="team",
    suffixes=["_A", "_B"]
).drop(
    ["scoreA", "scoreB", "teamA", "teamB", "team_A", "team_B", "wins_reg_2017/2018_A", "losses_reg_2017/2018_A", "wins_reg_2017/2018_B", "losses_reg_2017/2018_B"], 
    axis=1)

In [108]:
df_ml.head()

,teamAWinner,wins_reg_2015/2016_A,wins_reg_2016/2017_A,losses_reg_2015/2016_A,losses_reg_2016/2017_A,wins_reg_2015/2016_B,wins_reg_2016/2017_B,losses_reg_2015/2016_B,losses_reg_2016/2017_B
0,False,40,41,42,41,26,31,56,51
1,False,40,41,42,41,26,31,56,51
2,True,9,15,73,67,26,31,56,51
3,True,72,54,10,28,26,31,56,51
4,False,72,54,10,28,26,31,56,51


In [109]:
from sklearn.linear_model import LogisticRegression

In [110]:
lr = LogisticRegression()

In [111]:
from sklearn.model_selection import train_test_split

In [142]:
X_train, X_test, y_train, y_test = train_test_split(
    df_ml.drop("teamAWinner", axis=1), 
    df_ml["teamAWinner"]
)

lr.fit(X_train, y_train)

lr.score(X_test, y_test)

0.60064935064935066

# Average Win Margin per Team 

## Validierung der eigentlichen Cypher-Query

In [1]:
cypher = """
MATCH 
	(t:Team)-[:SCORED]->(s:Score)-[:IN_GAME]->(g:Game)<-[:IN_GAME]-(s2:Score)<-[:SCORED]-(t2:Team), (sea:Season)
WHERE 
    (g)-[:TOOK_PLACE_IN]->(sea)
AND
	sea.name = "2017/2018"
AND
	g.game_type = "regular_season"
WITH g, collect(t.name) as teams, collect(s.score) as scores
RETURN
	HEAD(teams) as teamA, HEAD(TAIL(scores)) as scoreA, HEAD(TAIL(teams)) as teamB, HEAD(scores) as scoreB
"""

In [5]:
data, cols = db.cypher_query(cypher)

In [6]:
df_matches = pd.DataFrame(data=data, columns=cols)

In [7]:
df_matches.head()

,teamA,scoreA,teamB,scoreB
0,Boston Celtics,80,Philadelphia 76ers,89
1,Charlotte Hornets,111,Toronto Raptors,129
2,Toronto Raptors,96,Detroit Pistons,91
3,Milwaukee Bucks,110,Toronto Raptors,129
4,Oklahoma City Thunder,105,Miami Heat,99


In [8]:
df_matches2 = df_matches

In [9]:
df_matches2["scoreMargin"] = abs(df_matches2["scoreA"]-df_matches2["scoreB"])

In [10]:
df_matches2.head()

,teamA,scoreA,teamB,scoreB,scoreMargin
0,Boston Celtics,80,Philadelphia 76ers,89,9
1,Charlotte Hornets,111,Toronto Raptors,129,18
2,Toronto Raptors,96,Detroit Pistons,91,5
3,Milwaukee Bucks,110,Toronto Raptors,129,19
4,Oklahoma City Thunder,105,Miami Heat,99,6


In [19]:
a = ["Golden State Warriors"]

In [20]:
df_onlyBOSmatches = df_matches2[(df_matches2['teamA'].isin(a)) | (df_matches2['teamB'].isin(a)) ]

In [21]:
df_onlyBOSmatches["scoreMargin"].mean()

13.463414634146341

## Actual Cypher Query

In [22]:
cypher = """
MATCH 
    (t:Team)-[:SCORED]->(s:Score)-[:IN_GAME]->(g:Game)<-[:IN_GAME]-(s2:Score)<-[:SCORED]-(t2:Team), (sea:Season)
WHERE 
    (g)-[:TOOK_PLACE_IN]->(sea)
RETURN 
    t.name as team, 
    avg(abs(s.score - s2.score)) as scoreMargin,
    sea.name as season
ORDER BY 
    team, season

"""

In [23]:
data, cols = db.cypher_query(cypher)

In [24]:
df_scoreMargin = pd.DataFrame(data=data, columns=cols)

In [25]:
df_scoreMargin.head()

,team,scoreMargin,season
0,Atlanta Hawks,11.717391,2015/2016
1,Atlanta Hawks,11.125000,2016/2017
2,Atlanta Hawks,10.865854,2017/2018
3,Boston Celtics,10.806818,2015/2016
4,Boston Celtics,10.380000,2016/2017


In [26]:
df_scoreMargin_test = df_scoreMargin.pivot(index="team", columns="season")

In [27]:
df_scoreMargin_test.head()

scoreMargin                      
season              2015/2016  2016/2017  2017/2018
team                                               
Atlanta Hawks       11.717391  11.125000  10.865854
Boston Celtics      10.806818  10.380000   9.693069
Brooklyn Nets       10.743902  12.195122  10.231707
Charlotte Hornets   11.449438  10.195122  11.768293
Chicago Bulls        9.865854  12.090909  12.231707

In [28]:
cols = []
for a, b in zip(df_scoreMargin_test.columns.get_level_values(0), df_scoreMargin_test.columns.get_level_values(1)):
    cols.append(a+"_"+b)

In [29]:
cols

['scoreMargin_2015/2016', 'scoreMargin_2016/2017', 'scoreMargin_2017/2018']

In [30]:
df_scoreMargin_test.columns = cols

In [32]:
df_scoreMargin_test.head()

,scoreMargin_2015/2016,scoreMargin_2016/2017,scoreMargin_2017/2018
team,,,
Atlanta Hawks,11.717391,11.125000,10.865854
Boston Celtics,10.806818,10.380000,9.693069
Brooklyn Nets,10.743902,12.195122,10.231707
Charlotte Hornets,11.449438,10.195122,11.768293
Chicago Bulls,9.865854,12.090909,12.231707
